In [25]:
# 라이브러리
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from graph import get_graph

In [26]:
path = "C:/Users/gihun/To_raise_a_child_well/data/gihun/dataset/"

In [27]:
sum_df = pd.read_excel(path+'전국데이터프레임2006~2021.xlsx',index_col=0)
sum_df

,실업률,이혼율,고령화비율,혼인건수,소비자물가지수,출산아수/가임기여성의수,아파트전세지수,미분양주택지수,은행예금지수,경제활동참가율,지가변동률
시점,,,,,,,,,,,
1999-06-01,5.6,0.001352,NaN,36243.0,61.349,NaN,NaN,NaN,275077.7,49.0,NaN
1999-07-01,5.6,0.001913,NaN,25464.0,61.140,NaN,NaN,NaN,271995.7,48.7,NaN
1999-08-01,5.0,0.002503,NaN,19217.0,61.765,NaN,NaN,NaN,288399.4,48.1,NaN
1999-09-01,4.2,0.002985,NaN,16485.0,61.921,NaN,NaN,NaN,304701.1,49.2,NaN
1999-10-01,4.0,0.002003,NaN,24713.0,62.389,NaN,NaN,NaN,301666.2,49.5,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2022-09-01,2.5,NaN,17.8,NaN,108.930,0.001486,102.8,102.8,1958348.3,55.1,NaN
2022-10-01,2.4,NaN,17.9,NaN,109.210,0.001486,101.4,101.4,1967290.0,55.0,NaN
2022-11-01,2.3,NaN,17.9,NaN,109.100,0.001486,99.0,99.0,1970661.1,55.1,NaN


In [28]:
# 독립변수,종속변수 설정
X = sum_df.drop(columns='출산아수/가임기여성의수').loc['2006-01-01':'2021-12-01'].values
y = sum_df['출산아수/가임기여성의수'].loc['2006-01-01':'2021-12-01'].values
X.shape, y.shape

((192, 10), (192,))

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [30]:
# 파라미터설정
param_grid = {'max_depth': [3, 4, 5],'learning_rate': [0.1, 0.01, 0.001],'n_estimators': [50, 100, 200]}

# 모델설정
cat = CatBoostRegressor()

# 그리드서치 파라미터 생성
grid = GridSearchCV(estimator=cat, param_grid=param_grid, cv=5)
grid.fit(X_train_scaled, y_train)
# 출력
print("Best hyperparameters: ", grid.best_params_)
print("Best score: ", grid.best_score_)

# 제일 좋은 것으로 다시돌리기
best_cat = CatBoostRegressor(max_depth=grid.best_params_['max_depth'],
                        learning_rate=grid.best_params_['learning_rate'],
                        n_estimators=grid.best_params_['n_estimators'])
best_cat.fit(X_train_scaled, y_train)

# 평가하기 
train_score = best_cat.score(X_train_scaled, y_train)
print("Train set score: ", train_score)
test_score = best_cat.score(X_test_scaled, y_test)
print("Test set score: ", test_score)

0:	learn: 0.0005235	total: 852us	remaining: 41.8ms
1:	learn: 0.0004928	total: 1.39ms	remaining: 33.3ms
2:	learn: 0.0004624	total: 1.86ms	remaining: 29.1ms
3:	learn: 0.0004375	total: 2.78ms	remaining: 31.9ms
4:	learn: 0.0004117	total: 3.39ms	remaining: 30.6ms
5:	learn: 0.0003890	total: 3.86ms	remaining: 28.3ms
6:	learn: 0.0003682	total: 4.39ms	remaining: 26.9ms
7:	learn: 0.0003488	total: 4.97ms	remaining: 26.1ms
8:	learn: 0.0003316	total: 5.44ms	remaining: 24.8ms
9:	learn: 0.0003186	total: 5.89ms	remaining: 23.6ms
10:	learn: 0.0003042	total: 6.62ms	remaining: 23.5ms
11:	learn: 0.0002892	total: 7.14ms	remaining: 22.6ms
12:	learn: 0.0002757	total: 7.93ms	remaining: 22.6ms
13:	learn: 0.0002646	total: 8.5ms	remaining: 21.9ms
14:	learn: 0.0002537	total: 9.07ms	remaining: 21.2ms
15:	learn: 0.0002455	total: 9.72ms	remaining: 20.7ms
16:	learn: 0.0002374	total: 10.2ms	remaining: 19.9ms
17:	learn: 0.0002296	total: 10.7ms	remaining: 18.9ms
18:	learn: 0.0002236	total: 11.1ms	remaining: 18ms
19:	lea

In [31]:
print("Best hyperparameters: ", grid.best_params_)
print("Best score: ", grid.best_score_)
print("Train set score: ", train_score)
print("Test set score: ", test_score)

Best hyperparameters:  {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
Best score:  0.910217708119742
Train set score:  0.9958884537699494
Test set score:  0.9565062475984256


In [32]:
# 결과 데이터프레임
result_df = pd.DataFrame(grid.cv_results_)
print(result_df[['params','mean_test_score','rank_test_score','split0_test_score','split1_test_score','split2_test_score','split3_test_score','split4_test_score']])

                                               params  mean_test_score  \
0   {'learning_rate': 0.1, 'max_depth': 3, 'n_esti...         0.872051   
1   {'learning_rate': 0.1, 'max_depth': 3, 'n_esti...         0.889967   
2   {'learning_rate': 0.1, 'max_depth': 3, 'n_esti...         0.902344   
3   {'learning_rate': 0.1, 'max_depth': 4, 'n_esti...         0.871041   
4   {'learning_rate': 0.1, 'max_depth': 4, 'n_esti...         0.893199   
5   {'learning_rate': 0.1, 'max_depth': 4, 'n_esti...         0.904259   
6   {'learning_rate': 0.1, 'max_depth': 5, 'n_esti...         0.882144   
7   {'learning_rate': 0.1, 'max_depth': 5, 'n_esti...         0.900257   
8   {'learning_rate': 0.1, 'max_depth': 5, 'n_esti...         0.910218   
9   {'learning_rate': 0.01, 'max_depth': 3, 'n_est...         0.424865   
10  {'learning_rate': 0.01, 'max_depth': 3, 'n_est...         0.638783   
11  {'learning_rate': 0.01, 'max_depth': 3, 'n_est...         0.801054   
12  {'learning_rate': 0.01, 'max_depth

In [38]:
best_cat.feature_importances_

array([ 3.52508096,  7.06083616, 20.72405354,  3.48328992, 23.0327664 ,
        6.2084338 ,  5.17716829, 18.78364188,  5.09109321,  6.91363584])

In [ ]:
# catboost에선 아파트전세지수, 혼인건수, 은행예금액